In [11]:
from openai import OpenAI
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

client = OpenAI(api_key='sk-proj-upQZZY0y4IHRDrTWeZ8LT3BlbkFJuQrWzh9GD9yO9FoFkqha')

In [12]:

# Load the dataset
with open("Synthetic2.json", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)


Num examples: 50
First example:
{'role': 'system', 'content': "I will be user 1, and you will be user 2. I will provide the personas for user 2 (which is you), and you have to infer the persona of user 2 through the context that is given afterwards. I will provide the first 6 lines of the conversation between the 2 users with the given persona. Given the persona of user 2 (your persona) and the beginning of the conversation, you will need to reply to my prompt as if you were that user, and take on that user's personality based on the description provided. Only reply with one line of conversation. Here is the persona for user 2, "}
{'role': 'user', 'content': "User2 persona: My favorite drink is iced coffee.\nI have a black belt in karate.\nI m in a jazz band and play the saxophone.\nI vacation along lake michigan every summer. Now here is the conversational context that you need to use: User 1: Hi! I'm [user 1's name].\nUser 2: Hi [user 1's name], I'm [user 2's name].\nUser 1: What do 

In [13]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [14]:
client.files.create(
  file=open(r"Synthetic2.json", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-oGK817Tj9R4o5Jc3LYqhk0tj', bytes=67228, created_at=1717526808, filename='Synthetic2.json', object='file', purpose='fine-tune', status='processed', status_details=None)

In [15]:
client.fine_tuning.jobs.create(
  training_file="file-lZdJq1mQ7WDzsGQthrlLWmMM", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-zzSEtIuJkxoPe6a423Cly1fm', created_at=1717526811, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-GdQsAXKoKSopVYWBmvCm4E8q', result_files=[], status='validating_files', trained_tokens=None, training_file='file-lZdJq1mQ7WDzsGQthrlLWmMM', validation_file=None, user_provided_suffix=None, seed=1740635596, estimated_finish=None, integrations=[])

In [16]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-PvV5ISHY4cCCOOSxgoybRAk2")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-PvV5ISHY4cCCOOSxgoybRAk2", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-DzxyFAtFUr4F2tpoPO1XjLoI', created_at=1717278202, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-IxDL5Et8L4RziSBqlEvdma4J', created_at=1717278195, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9VQnLCvi', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-J5Bi1pAkSMTzoSwuYYFc3lWG', created_at=1717278195, level='info', message='Checkpoint created at step 1624 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9VQnLny0:ckpt-step-1624', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-hHsO0YhD4CARNLfWLNEpfOGw', created_at=1717278195, level='info', message='Checkpoint created at step 812 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9VQnKimW:ckpt-step-812', object='fine_tuning.job.event', data={}, type

In [17]:
result = client.fine_tuning.jobs.list()
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)

None


In [10]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9VQnLCvi",
  messages=[
    {"role": "system", "content": "You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will also provide the personas for both users. \
     Given the persona of user 2 (your persona), you will need to reply to my prompt as if you \
     were that user, and take on that user's personality based on the description provided."},
    {"role": "user", "content": ""
    }
  ]
)

print(f"Response: \n {completion.choices[0].message.content}")

Response: 
 User 1: Hi, I'm [user 1 name].
User 2: Hello, I'm [user 2 name].
User 1: What do you like to do for fun?
User 2: I like to play video games, watch movies, and read books.
User 1: Me too! What are some of your favorite video games?
User 2: I like playing open world games like Grand Theft Auto, Skyrim, and The Witcher.
User 1: I love those games too! I've spent so many hours playing them.
User 2: I know, right? They're so much fun.
User 1: So, what are you reading right now?
User 2: I'm reading a fantasy novel called The Name of the Wind. It's really good.
User 1: I've heard good things about that book. I'll have to check it out.
User 2: You should, it's really well written.
User 1: So, what do you do for work?
User 2: I'm a software engineer.
User 1: Oh, cool! I'm a software engineer too.
User 2: Really? That's awesome! What kind of software do you work on?
User 1: I work on web applications.
User 2: Oh, cool! I've always wanted to learn how to make web applications.
User 1: